In [23]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2:latest",
    temperature=0.7
)


In [24]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一位專業的餐廳評論家，擅長分析參的優缺點。"),
        ("human","請分析這家餐廳：{restaurant_name}")
    ]
)

In [25]:
pros_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一位專業的餐廳評論家，擅長分析參的優點。"),
        ("human","根據餐廳資訊：{restaurant_info}，請列出這家餐廳的優點。")
    ]
)

pros_branch_chain = (
    pros_template
    | model
    | StrOutputParser()
)

In [26]:
cons_template = ChatPromptTemplate.from_messages(
    [
        ("system","你是一位專業的餐廳評論家，擅長分析參的缺點。"),
        ("human","根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。")
    ]
)

cons_branch_chain = (
    cons_template
    | model
    | StrOutputParser()
)

In [27]:
# 定義結果合併函數
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

print("結果合併函數已定義：combine_pros_cons")


結果合併函數已定義：combine_pros_cons


In [28]:
chain = (
    prompt_template 
    | model
    | StrOutputParser()
    | RunnableLambda(lambda x:{"restaurant_info":x})
    | RunnableParallel(
        pros = pros_branch_chain,
        cons = cons_branch_chain
    )
    | RunnableLambda(lambda x: combine_pros_cons(x['pros'],x['cons']))
)

In [29]:
restaurant = """
這是一家美式餐廳，有明亮的裝潢，提供各式漢堡、薯條和飲料，氣氛輕鬆適合聚會。
"""

result = chain.invoke({"restaurant_name":restaurant})

print(result)


🍽️ 餐廳評論分析報告

✅ 優點分析：
根據評論家的分析，這家美式餐廳的優點如下：

1. **明亮的裝潢**：餐廳的現代風格是吸引人的一大優勢。
2. **多樣化的菜單**：提供各式漢堡、薯條和飲料，滿足不同口味和需求。
3. **適合聚會**：餐廳的氣氛輕鬆，適合為朋友聚集來吃飯。

❌ 缺點分析：
根據提供的資訊，我們可以列出這家餐廳的缺點如下：

1.  **無原創性**： menu 內容似乎並不 TOO 原創，有些菜品可能與其他餐廳有所重疊。
2.  **裝潢上的過度簡化**：雖然明亮的裝潢是優點之一，但是否真的足夠讓客戶感受到特別呢？在現代餐廳中，通常需要更多的細節和創意來吸引客戶注意力。
3.  **沒有特殊的賣点**：雖然餐廳適合聚會，但它並沒有什麼特別的 selling point，可以令客戶記得這家餐廳。

📊 總結：這是一個平衡的餐廳評論分析

